<a href="https://colab.research.google.com/github/YuvalShafriri/CBSA2.0/blob/main/graph_lla.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install --upgrade --quiet  langchain langchain-community langchain-openai langchain-experimental neo4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.4/193.4 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.0/203.0 kB 15.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.5/287.5 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.0/113.0 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.3/268.3 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 58.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.5 MB/s eta 0:00:00
     ━━━

In [9]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = "sk-"
# getpass.getpass()

# Uncomment the below to use LangSmith. Not required.
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()
# os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [10]:


from langchain_community.graphs import Neo4jGraph

os.environ["NEO4J_URI"] = "neo4j+s://fafcd591.databases.neo4j.io"
#os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = ""

graph = Neo4jGraph()

In [11]:


from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0, model_name="gpt-4-0125-preview")

llm_transformer = LLMGraphTransformer(llm=llm)

In [12]:
from langchain_core.documents import Document

text1 = """
Marie Curie, was a Polish and naturalised-French physicist and chemist who conducted pioneering research on radioactivity.
She was the first woman to win a Nobel Prize, the first person to win a Nobel Prize twice, and the only person to win a Nobel Prize in two scientific fields.
Her husband, Pierre Curie, was a co-winner of her first Nobel Prize, making them the first-ever married couple to win the Nobel Prize and launching the Curie family legacy of five Nobel Prizes.
She was, in 1906, the first woman to become a professor at the University of Paris.
"""
text = """
Issachar plateau is characterized as an isolated marginal area, which was sparsely populated in antiquity and about which historical information is scarce. However, during the Byzantine period, settlement flourished in the area, and Gevul is part of the settlements along the Issachar stream and its tributaries, and part of a concentration of sites where synagogue items were found. It is mentioned in the Talmud and medieval travelers' writings, but the site has not been thoroughly surveyed and archaeological information about the site is limited. Remains on the surface are meager, but architectural items, ashlar stones, a wall, and a leveled surface suggest a public building. The uniqueness of the site is two identical single stones decorated with Jewish symbols of a menorah, Lulav and Etrog, a "Machta" (a kind of dustpan) and a Shofar that may have been incorporated into the pulpit of the Holy Ark. The stones were removed from the site and only a few architectural elements remained visible in the site.
"""


In [15]:
documents = [Document(page_content=text)]
graph_documents = llm_transformer.convert_to_graph_documents(documents)
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

Nodes:[Node(id='Issachar Plateau', type='Place'), Node(id='Gevul', type='Place'), Node(id='Byzantine Period', type='Time period'), Node(id='Talmud', type='Text'), Node(id="Medieval Travelers' Writings", type='Text'), Node(id='Menorah', type='Artifact'), Node(id='Lulav', type='Artifact'), Node(id='Etrog', type='Artifact'), Node(id='Machta', type='Artifact'), Node(id='Shofar', type='Artifact'), Node(id='Holy Ark', type='Artifact')]
Relationships:[Relationship(source=Node(id='Issachar Plateau', type='Place'), target=Node(id='Gevul', type='Place'), type='INCLUDES'), Relationship(source=Node(id='Issachar Plateau', type='Place'), target=Node(id='Byzantine Period', type='Time period'), type='TIME_PERIOD_OF_ACTIVITY'), Relationship(source=Node(id='Gevul', type='Place'), target=Node(id='Talmud', type='Text'), type='MENTIONED_IN'), Relationship(source=Node(id='Gevul', type='Place'), target=Node(id="Medieval Travelers' Writings", type='Text'), type='MENTIONED_IN'), Relationship(source=Node(id='Ge

In [16]:
graph.add_graph_documents(graph_documents)